In [31]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error, classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import RidgeClassifier

### # Image Augmentation
#### - Add more variations to training images
#### - Techniques: flip, rotate, crop, noise
#### - Libraries: OpenCV, NumPy

In [32]:
input_dir = "./data/prepared-2"
output_dir = "./data/prepared-train"
os.makedirs(output_dir, exist_ok=True)

In [33]:
# count images
image_paths = glob(os.path.join(input_dir, "*.jpg"))
print(f"Total images: {len(image_paths)}")

จำนวนภาพทั้งหมด: 75


In [34]:
def augment_image(img):
    augmented = []

    # Flip
    augmented.append(("flip_h", cv2.flip(img, 1)))   # horizontal
    augmented.append(("flip_v", cv2.flip(img, 0)))   # vertical

    # Rotate
    (h, w) = img.shape[:2]
    M = cv2.getRotationMatrix2D((w//2, h//2), 15, 1.0)
    augmented.append(("rot15", cv2.warpAffine(img, M, (w, h))))

    # Crop (random crop)
    cropped = img[5:h-5, 5:w-5]
    cropped = cv2.resize(cropped, (w, h))
    augmented.append(("crop", cropped))

    # Noise
    noise = np.random.normal(0, 25, img.shape).astype(np.uint8)
    noisy = cv2.add(img, noise)
    augmented.append(("noise", noisy))

    return augmented

# -----------------------------
# Process Images
# -----------------------------
image_paths = glob(os.path.join(input_dir, "*.jpg"))

for path in image_paths:
    label = os.path.basename(os.path.dirname(path))
    fname = os.path.splitext(os.path.basename(path))[0]

    # เตรียมโฟลเดอร์ class
    out_class_dir = output_dir
    os.makedirs(out_class_dir, exist_ok=True)

    # โหลดภาพ
    img = cv2.imread(path)

    # save original
    cv2.imwrite(os.path.join(out_class_dir, f"{fname}_orig.jpg"), img)

    # save augmentations
    for aug_name, aug_img in augment_image(img):
        out_path = os.path.join(out_class_dir, f"{fname}_{aug_name}.jpg")
        cv2.imwrite(out_path, aug_img)

print("✅ เสร็จสิ้น: Augmented images saved to", output_dir)

✅ เสร็จสิ้น: Augmented images saved to ./data/prepared-train


### # Prepare Data for Training

In [35]:
input_dir = "./data/prepared-train"
image_paths = glob(os.path.join(input_dir, "*.jpg"))
print(f"Total images: {len(image_paths)}")

Total images: 450
